Just setting the environment up, assuming all module have been installed prior.

In [16]:
import requests, json, time, pathlib, yaml
import pandas as pd
import matplotlib.pyplot as plt 
from cli_badges import badge

In [17]:
def RealTimeSharePrice (stock_symbol, api_key) :
    '''This function loads the current shareprice from alphavantage'''
    base_url = "https://www.alphavantage.co/query?function=GLOBAL_QUOTE"
    main_url = base_url + "&symbol=" + stock_symbol + "&apikey=" + api_key
    req_ob = requests.get(main_url)                     #get method of requests module & return response object
    result = req_ob.json()                              #json method return json format data into python dictionary data typ & result contains list of nested dictionaries
    if debug == True : 
        print("Result before parsing the json data :\n", result)
        print("Parsed as :\n", result["Global Quote"])
        print("Lenght ", len(result["Global Quote"]))
    if len(result["Global Quote"]) > 0: 
        share_price = float(result["Global Quote"]['05. price'])
        share_change = float(result["Global Quote"]["09. change"])
        if share_change >= 0 :
            messagebg = 'green'
        else :
            messagebg = 'red'
        label = result["Global Quote"]["01. symbol"]
        price = result["Global Quote"]["05. price"]
        change = result["Global Quote"]["09. change"]
        PriceBadge = badge(label, price, messagebg='blue',messagecolor='black')
        ChangeBadge = badge(label, change, messagebg=messagebg, messagecolor='black')
        print(PriceBadge, ChangeBadge, "\n")
        if debug == True : 
            print("Realtime share price for", result["Global Quote"]["01. symbol"], "price", result["Global Quote"]["05. price"], "change", result["Global Quote"]["09. change"])
    else:
        share_price = float(0)
    return(share_price)

In [7]:
def AskInputShareCode (): 
    stock_symbol =input("Please enter stock symbol to test on AlphaVantage :")
    return (stock_symbol)

In [8]:
def ProcessYAML (yaml_file) :
    '''This function opens the yaml file and returns the data object'''
    with open(yaml_file) as f:
        y_data = yaml.load(f, Loader=yaml.FullLoader)
        debug = y_data['debug']
        if debug == True : print("YAML file:\n", y_data)
    return (y_data)    

In [14]:
yaml_data = ProcessYAML('alpha.yaml')                                     #yaml settings are global variables
debug = yaml_data['debug']                                                #debug mode?
plot = yaml_data['plot']                                                  #plot mode?  
copy_to_file = yaml_data['copy_to_file']                                  #file output
api_key = yaml_data['api_key_dev']

In [18]:
stock_symbol = AskInputShareCode() 
if debug == True : print (api_key)
share_price = RealTimeSharePrice(stock_symbol, api_key)
#intra_day_price_df = IntraDayPrice( stock_symbol, api_key)
#hist_share_price_df = HistoricSharePrice (stock_symbol, api_key)
#if plot == True : 
    #PlotIntraDayPrice(stock_symbol, intra_day_price_df)
    #PlotHistoricSharePrice(stock_symbol, hist_share_price_df)
    #plt.show()
print("===========")

 T  30.0400   T  0.1400  

